In [1]:
import numpy as np
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from practical_b_1 import ImageDataset,SortedImageDataset
from load_mnist import load_mnist
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
criterion = nn.CrossEntropyLoss()
batch_size = 64
c=10
EPOCHS = 5
input_size = 28*28
output_size=10

In [3]:
class Network(nn.Module):
    def __init__(self,num_neurons,input_size,output_size):
        super().__init__()
        hidden1 = 50
        hidden2 = 20
        hidden3 = 70
        self.conv = nn.Sequential(
          nn.Linear(input_size,hidden1),
          nn.ReLU(),
          nn.Linear(hidden1,hidden2),
          nn.ReLU(),
          nn.Linear(hidden2,hidden3),
          nn.ReLU(),
          nn.Linear(hidden3, output_size)
        )
    
    def forward(self, x):
        return nn.Softmax(dim=0)(self.conv(x))

train the model

In [4]:
def train(model, dataloader: DataLoader, optimizer):
    
    epoch_loss=0.0
    for batch, (images, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(images.view(-1, input_size))
        output=output.float()
        loss=criterion(output.squeeze(),labels.long())
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    epoch_loss = epoch_loss / len(dataloader)
    print('loss :{}'.format(loss))    


train loop

In [5]:
def trainloop(dataloader, model, optimizer,EPOCHS):
    for epoch in range(EPOCHS):
        train(dataloader=dataloader, model=model, optimizer=optimizer)

Intialize model,optimizer,trainDataset,testDataset,traindataloader,testdataloader

In [6]:
model = Network(num_neurons=(20,10,7),input_size=input_size,output_size=output_size)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [7]:
def test_(test_loader:DataLoader,EPOCHS:int,model):
    model.eval()
    for epoch in range(EPOCHS):                                             
        with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    outputs = model(images.view(-1, input_size))
                    _, predicted = torch.max(outputs.data, 1)
                    # total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    
    
                print('Accuracy: %d %%' % (100 * correct / (len(test_loader.dataset)))) 

### 2.1
#### a
Train the classes separately, i.e. filter the dataset and train for five epochs
on all images depicting a 0, then on all images depicting a 1, and so on.

In [8]:
classes = range(c)
for label in classes:
    model_b = Network(num_neurons=(20,10,7),input_size=input_size,output_size=output_size)
    trainDataset = ImageDataset(type='training', label=label)
    data_loader = DataLoader(trainDataset, batch_size=batch_size,shuffle=False, drop_last=False)
    trainloop(dataloader=data_loader, model=model,
              optimizer=optimizer, EPOCHS=5)
    testingDataset = ImageDataset(type='testing', label=label)
    test_loader = DataLoader(testingDataset, batch_size=batch_size,shuffle=True, drop_last=False)
    print('test label: {}'.format(label))
    test_(model=model_b,EPOCHS=EPOCHS,test_loader=test_loader)          

loss :2.3045785427093506
loss :2.304482936859131
loss :2.304396867752075
loss :2.3043196201324463
loss :2.304248571395874
test label: 0
Accuracy: 10 %
Accuracy: 10 %
Accuracy: 10 %
Accuracy: 9 %
Accuracy: 10 %
loss :2.303076982498169
loss :2.3030734062194824
loss :2.303069591522217
loss :2.303065538406372
loss :2.3030614852905273
test label: 1
Accuracy: 6 %
Accuracy: 6 %
Accuracy: 6 %
Accuracy: 6 %
Accuracy: 5 %
loss :2.3037185668945312
loss :2.30366587638855
loss :2.3036134243011475
loss :2.303558349609375
loss :2.303504228591919
test label: 2
Accuracy: 4 %
Accuracy: 3 %
Accuracy: 4 %
Accuracy: 4 %
Accuracy: 4 %
loss :2.303084135055542
loss :2.3030688762664795
loss :2.30305552482605
loss :2.303044557571411
loss :2.3030359745025635
test label: 3
Accuracy: 10 %
Accuracy: 8 %
Accuracy: 9 %
Accuracy: 9 %
Accuracy: 9 %
loss :2.3034725189208984
loss :2.3034441471099854
loss :2.303419828414917
loss :2.3033971786499023
loss :2.3033761978149414
test label: 4
Accuracy: 9 %
Accuracy: 9 %
Accurac

### 2.1
#### b
Sort the images by class and train them in order for 25 epochs. Make sure
that the DataLoader is not shuffling the data.

In [9]:
sortedDataSet=SortedImageDataset('training')
sortedDataSetLoader = DataLoader(sortedDataSet, batch_size=batch_size,
                                              shuffle=False, drop_last=False)
model_b = Network(num_neurons=(20,10,7),input_size=input_size,output_size=output_size)
sortedEpoch=25
trainloop(dataloader=sortedDataSetLoader, model=model_b,
              optimizer=optimizer, EPOCHS=sortedEpoch)    

ggg=SortedImageDataset(type='testing')
test_loader = DataLoader(ggg, batch_size=batch_size,shuffle=True, drop_last=False)
test_(model=model_b,EPOCHS=sortedEpoch,test_loader=test_loader)
                                         

loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
loss :2.3061227798461914
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %
Accuracy: 2 %


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8d525717-702e-4285-81d5-cd5945dff109' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>